In [ ]:
!git clone https://github.com/ggerganov/llama.cpp

fatal: destination path 'llama.cpp' already exists and is not an empty directory.


In [ ]:
!cd llama.cpp && LLAMA_CUBLAS=1 make && pip install -r requirements/requirements-convert-hf-to-gguf.txt

I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_CUDA -I/usr/local/cuda/include -I/usr/local/cuda/targets/x86_64-linux/include  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_CUDA -I/usr/local/cuda/include -I/usr/local/cuda/targets/x86_64-linux/include 
I NVCCFLAGS: -std=c++11 -O3 -use_fast_math --forward-unkn

In [ ]:
from huggingface_hub import snapshot_download

In [ ]:
model_name = "pmking27/PrathameshLLM-7B"

In [ ]:
methods = ['q2_k',' iq3_xs','q3_k_s','iq3_s','iq3_m','q3_k_m','q3_k_l','iq4_xs','q4_0','q4_xs','iq4_nl','q4_k_m','q4_k_s','q5_0','q5_k_s','q5_k_m','q6_k','q8_0']

In [ ]:
base_model = "./original_model/"
quantized_path = "./quantized_model/"

In [ ]:
snapshot_download(repo_id=model_name, local_dir=base_model , local_dir_use_symlinks=False)
original_model = quantized_path+'/FP16.gguf'

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/3.46k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

In [ ]:
!mkdir ./quantized_model/

mkdir: cannot create directory ‘./quantized_model/’: File exists


In [ ]:
!python llama.cpp/convert-hf-to-gguf.py ./original_model/ --outtype f16 --outfile ./quantized_model/FP16.gguf

Loading model: original_model
gguf: This GGUF file is for Little Endian only
Set model parameters
Set model tokenizer
gguf: Setting special token type bos to 2
gguf: Setting special token type eos to 1
gguf: Setting special token type unk to 3
gguf: Setting special token type pad to 0
gguf: Setting add_bos_token to True
gguf: Setting add_eos_token to False
Exporting model to 'quantized_model/FP16.gguf'
gguf: loading model part 'model-00001-of-00002.safetensors'
token_embd.weight, n_dims = 2, torch.float16 --> float32
blk.0.attn_norm.weight, n_dims = 1, torch.float16 --> float32
blk.0.ffn_down.weight, n_dims = 2, torch.float16 --> float32
blk.0.ffn_gate.weight, n_dims = 2, torch.float16 --> float32
blk.0.ffn_up.weight, n_dims = 2, torch.float16 --> float32
blk.0.ffn_norm.weight, n_dims = 1, torch.float16 --> float32
blk.0.attn_k.weight, n_dims = 2, torch.float16 --> float32
blk.0.attn_output.weight, n_dims = 2, torch.float16 --> float32
blk.0.attn_q.weight, n_dims = 2, torch.float16 -->

In [ ]:
import os

In [ ]:
for m in methods:
    qtype = f"{quantized_path}/{m.upper()}.gguf"
    os.system("./llama.cpp/quantize "+quantized_path+"/FP16.gguf "+qtype+" "+m)

In [ ]:
! ./llama.cpp/main -m ./quantized_model/Q3_K_M.gguf -n 90 --repeat_penalty 1.0 --color -i -r "User:" -f llama.cpp/prompts/chat-with-bob.txt

Log start
main: build = 2607 (c666ba26)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: seed  = 1712253942
llama_model_loader: loaded meta data with 23 key-value pairs and 164 tensors from ./quantized_model/Q3_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = original_model
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                     gemma.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          gemma.block_count u32              = 18
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32              = 16384
llama_model_loader: - kv   

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from huggingface_hub import HfApi, HfFolder, create_repo, upload_file

In [ ]:
model_path = "./quantized_model/Q4_K_M.gguf" # Your model's local path
repo_name = "PrathameshLLM-7B"  # Desired HF Hub repository name
repo_url = create_repo(repo_name, private=False)

In [ ]:
api = HfApi()
api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo="PrathameshLLM-7B.Q4_K_M.gguf",
    repo_id="pmking27/PrathameshLLM-7B-GGUF",
    repo_type="model",
)

Q4_K_M.gguf:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/pmking27/PrathameshLLM-7B-GGUF/commit/982c43c5b583ff060d8f5c62a0aca2eafae3f477', commit_message='Upload Q4_K_M.gguf with huggingface_hub', commit_description='', oid='982c43c5b583ff060d8f5c62a0aca2eafae3f477', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model_path = "./quantized_model/Q2_K.gguf" # Your model's local path
repo_name = "PrathameshLLM-7B"  # Desired HF Hub repository name
# repo_url = create_repo(repo_name, private=False)
api = HfApi()
api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo="PrathameshLLM-7B.Q2_K.gguf",
    repo_id="pmking27/PrathameshLLM-7B-GGUF",
    repo_type="model",
)

In [ ]:
model_path = "./quantized_model/IQ3_XS.gguf" # Your model's local path
repo_name = "PrathameshLLM-7B"  # Desired HF Hub repository name
repo_url = create_repo(repo_name, private=False)
api = HfApi()
api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo="PrathameshLLM-7B.IQ3_XS.gguf",
    repo_id="pmking27/PrathameshLLM-7B-GGUF",
    repo_type="model",
)

In [ ]:
model_path = "./quantized_model/Q3_K_S.gguf" # Your model's local path
repo_name = "PrathameshLLM-7B"  # Desired HF Hub repository name
# repo_url = create_repo(repo_name, private=False)
api = HfApi()
api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo="PrathameshLLM-7B.Q3_K_S.gguf",
    repo_id="pmking27/PrathameshLLM-7B-GGUF",
    repo_type="model",
)

Q3_K_S.gguf:   0%|          | 0.00/1.29G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/pmking27/PrathameshLLM-7B-GGUF/commit/9e2a803a29a733dece3509354b0e81dc8c1911e9', commit_message='Upload Q3_K_S.gguf with huggingface_hub', commit_description='', oid='9e2a803a29a733dece3509354b0e81dc8c1911e9', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model_path = "./quantized_model/IQ3_S.gguf" # Your model's local path
repo_name = "PrathameshLLM-7B"  # Desired HF Hub repository name
# repo_url = create_repo(repo_name, private=False)
api = HfApi()
api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo="PrathameshLLM-7B.IQ3_S.gguf",
    repo_id="pmking27/PrathameshLLM-7B-GGUF",
    repo_type="model",
)

CommitInfo(commit_url='https://huggingface.co/pmking27/PrathameshLLM-7B-GGUF/commit/0909d03e20562ab59a46278fe37900022bca63d4', commit_message='Upload IQ3_S.gguf with huggingface_hub', commit_description='', oid='0909d03e20562ab59a46278fe37900022bca63d4', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model_path = "./quantized_model/IQ3_M.gguf" # Your model's local path
repo_name = "PrathameshLLM-7B"  # Desired HF Hub repository name
# repo_url = create_repo(repo_name, private=False)
api = HfApi()
api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo="PrathameshLLM-7B.IQ3_M.gguf",
    repo_id="pmking27/PrathameshLLM-7B-GGUF",
    repo_type="model",
)

CommitInfo(commit_url='https://huggingface.co/pmking27/PrathameshLLM-7B-GGUF/commit/bafd259733fa3c8e781f2a5141f4876eebdf1099', commit_message='Upload PrathameshLLM-7B-GGUF.IQ3_M.gguf with huggingface_hub', commit_description='', oid='bafd259733fa3c8e781f2a5141f4876eebdf1099', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model_path = "./quantized_model/Q3_K_M.gguf" # Your model's local path
repo_name = "PrathameshLLM-7B"  # Desired HF Hub repository name
# repo_url = create_repo(repo_name, private=False)
api = HfApi()
api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo="PrathameshLLM-7B.Q3_K_M.gguf",
    repo_id="pmking27/PrathameshLLM-7B-GGUF",
    repo_type="model",
)

In [ ]:
model_path = "./quantized_model/Q3_K_L.gguf" # Your model's local path
repo_name = "PrathameshLLM-7B"  # Desired HF Hub repository name
# repo_url = create_repo(repo_name, private=False)
api = HfApi()
api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo="PrathameshLLM-7B.Q3_K_L.gguf",
    repo_id="pmking27/PrathameshLLM-7B-GGUF",
    repo_type="model",
)

In [ ]:
model_path = "./quantized_model/IQ4_XS.gguf" # Your model's local path
repo_name = "PrathameshLLM-7B"  # Desired HF Hub repository name
# repo_url = create_repo(repo_name, private=False)
api = HfApi()
api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo="PrathameshLLM-7B.IQ4_XS.gguf",
    repo_id="pmking27/PrathameshLLM-7B-GGUF",
    repo_type="model",
)

CommitInfo(commit_url='https://huggingface.co/pmking27/PrathameshLLM-7B-GGUF/commit/392f9a7834989e9eb0e2fb09ce92e892b3882b79', commit_message='Upload PrathameshLLM-7B-GGUF.IQ4_XS.gguf with huggingface_hub', commit_description='', oid='392f9a7834989e9eb0e2fb09ce92e892b3882b79', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model_path = "./quantized_model/Q4_0.gguf" # Your model's local path
repo_name = "PrathameshLLM-7B"  # Desired HF Hub repository name
# repo_url = create_repo(repo_name, private=False)
api = HfApi()
api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo="PrathameshLLM-7B.Q4_0.gguf",
    repo_id="pmking27/PrathameshLLM-7B-GGUF",
    repo_type="model",
)

In [ ]:
model_path = "./quantized_model/Q4_K_S.gguf" # Your model's local path
repo_name = "PrathameshLLM-7B"  # Desired HF Hub repository name
# repo_url = create_repo(repo_name, private=False)
api = HfApi()
api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo="PrathameshLLM-7B.Q4_K_S.gguf",
    repo_id="pmking27/PrathameshLLM-7B-GGUF",
    repo_type="model",
)

CommitInfo(commit_url='https://huggingface.co/pmking27/PrathameshLLM-7B-GGUF/commit/4d151f3d7b21d42054c2fa52d9f96858f161b980', commit_message='Upload PrathameshLLM-7B-GGUF.Q4_K_S.gguf with huggingface_hub', commit_description='', oid='4d151f3d7b21d42054c2fa52d9f96858f161b980', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model_path = "./quantized_model/IQ4_NL.gguf" # Your model's local path
repo_name = "PrathameshLLM-7B"  # Desired HF Hub repository name
# repo_url = create_repo(repo_name, private=False)
api = HfApi()
api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo="PrathameshLLM-7B.IQ4_NL.gguf",
    repo_id="pmking27/PrathameshLLM-7B-GGUF",
    repo_type="model",
)

CommitInfo(commit_url='https://huggingface.co/pmking27/PrathameshLLM-7B-GGUF/commit/bc8936302772051f8e28665af48916695d578c45', commit_message='Upload PrathameshLLM-7B-GGUF.IQ4_NL.gguf with huggingface_hub', commit_description='', oid='bc8936302772051f8e28665af48916695d578c45', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model_path = "./quantized_model/Q5_0.gguf" # Your model's local path
repo_name = "PrathameshLLM-7B"  # Desired HF Hub repository name
# repo_url = create_repo(repo_name, private=False)
api = HfApi()
api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo="PrathameshLLM-7B.Q5_0.gguf",
    repo_id="pmking27/PrathameshLLM-7B-GGUF",
    repo_type="model",
)

CommitInfo(commit_url='https://huggingface.co/pmking27/PrathameshLLM-7B-GGUF/commit/272437424c6638d1785043ed5c2ba1e9ed057c85', commit_message='Upload PrathameshLLM-7B-GGUF.Q5_0.gguf with huggingface_hub', commit_description='', oid='272437424c6638d1785043ed5c2ba1e9ed057c85', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model_path = "./quantized_model/Q5_K_S.gguf" # Your model's local path
repo_name = "PrathameshLLM-7B"  # Desired HF Hub repository name
# repo_url = create_repo(repo_name, private=False)
api = HfApi()
api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo="PrathameshLLM-7B.Q5_K_S.gguf",
    repo_id="pmking27/PrathameshLLM-7B-GGUF",
    repo_type="model",
)

In [ ]:
model_path = "./quantized_model/Q5_K_M.gguf" # Your model's local path
repo_name = "PrathameshLLM-7B"  # Desired HF Hub repository name
# repo_url = create_repo(repo_name, private=False)
api = HfApi()
api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo="PrathameshLLM-7B.Q5_K_M.gguf",
    repo_id="pmking27/PrathameshLLM-7B-GGUF",
    repo_type="model",
)

In [ ]:
model_path = "./quantized_model/Q6_K.gguf" # Your model's local path
repo_name = "PrathameshLLM-7B"  # Desired HF Hub repository name
# repo_url = create_repo(repo_name, private=False)
api = HfApi()
api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo="PrathameshLLM-7B.Q6_K.gguf",
    repo_id="pmking27/PrathameshLLM-7B-GGUF",
    repo_type="model",
)

In [ ]:
model_path = "./quantized_model/Q8_0.gguf" # Your model's local path
repo_name = "PrathameshLLM-7B"  # Desired HF Hub repository name
# repo_url = create_repo(repo_name, private=False)
api = HfApi()
api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo="PrathameshLLM-7B.Q8_0.gguf",
    repo_id="pmking27/PrathameshLLM-7B-GGUF",
    repo_type="model",
)